In [1]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt

In [5]:
mkdir -p ./3.delta_heatmap_piecharts

For 3 “the delta heatmap”, it is not only for peaks with more than 1.2-fold changes. Basically you will directly use the **pair-wise comparison signal files you sent to me on July 7th** to generate the delta heatmap, they are named as “**293_Ts_WT.merge.sig.anno.pval**”. Based on that table, you use FC1.2 as cutoff to group peaks into Up, No change, and Down subgroups. Just a reminder, since now we have already used local lamda and input to help with the peak calling, no need to compare to input signal to remove non-specific peaks any more. Then plot them for **peak centered 5 kb** plus/minus regions, similar to the delta heatmap you generated earlier (one example attached here). For the delta map part, use the color scheme Liling suggested.

* use the 293_Ts_WT.merge.sig.anno.pval in ./res_avg_sig
* split the files into three groups **Note** the previous generated group (293.sig.anno) is it the same as the computeMatrix?
* computeMatrix use the "\t" seperated bed file


# 1. Generate the delta heatmap

In [2]:
os.chdir('/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/')

In [4]:
treats = ['T1', 'T2', 'T3', 'Y78A']
ctrl = 'ENL'
# sigpath: store the previous generated merged peaks signal for T and wt
sigpath = './2.Re_do_with_lambda_macs/' 
outpath = './3.delta_heatmap_piecharts/'
bwpath = '/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/'
exp = 'HK2'
treat = treats[0]

> generate the bed file and use computeMatrix to generate the bin signal

In [5]:
!pwd
sigfile = f'{sigpath}{exp}_{treat}_WT.merge.sig.anno.pval.txt'
!head $sigfile -n2

/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie
chr	start	end	width	HK2_F_T1_treat	HK2_F_ENL_treat	HK2_ctrl	within_genebody	nearest_TSS	distance	logFC	PValue
chr1	804921	805827	906.0	10.561810154525388	11.479028697571746	2.8454746136865343	FAM41C	none	none	-0.12	0.978


Use the previous bigWig Generated signal to rank the split the peaks into three groups: upper, lower, nodiff
Then, use this peak to extract the signal from bw using deeptools computeMatrix
Finally, use the matrix to do the heatmap plot

Bin size: set from 50 to 100

In [10]:
for treat in treats:
    #1. extract the first three columns as the bed file
    sigfile = f'{sigpath}{exp}_{treat}_WT.merge.sig.anno.pval.txt'
    line = "awk \'BEGIN{OFS=\"\\t\"}{if (NR > 1) print $1, $2, $3,NR-2,\".\", \"+\"}\'" + f' {sigfile} > {outpath}{treat}.bed'
    print(line + '\n****')
    os.system(line)
    # 2. use the bed file to extract the binning signal (center -5k~5k)
    bw_files = [
            glob.glob(bwpath + exp + '*' + tmp + '*.bw')[0]
            for tmp in [treat, ctrl]
    ]
    bw_files_format = len(bw_files) * "{} "
    bw_files_format = bw_files_format.format(*bw_files)
    bed_file_format = f'{outpath}{treat}.bed'
    command = "computeMatrix reference-point --referencePoint center -b 5000 -a 5000 -bs 100 --missingDataAsZero -R {0} -S {1} --skipZeros -o {2}.scale.gz --outFileNameMatrix {3}.scale.txt ".format(
    bed_file_format, bw_files_format,
    outpath + exp + '_' + treat + '_' + ctrl,
    outpath + exp + '_' + treat + '_' + ctrl)
    print(command + '\n\n')
    os.system(command)
    os.system(
        'rm {}.scale.gz'.format(outpath + exp + '_' + treat + '_' + ctrl))   
    
    

awk 'BEGIN{OFS="\t"}{if (NR > 1) print $1, $2, $3,NR-2,".", "+"}' ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.sig.anno.pval.txt > ./3.delta_heatmap_piecharts/T1.bed
****
computeMatrix reference-point --referencePoint center -b 5000 -a 5000 -bs 100 --missingDataAsZero -R ./3.delta_heatmap_piecharts/T1.bed -S /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T1_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_ENL_treat.nsp.bw  --skipZeros -o ./3.delta_heatmap_piecharts/HK2_T1_ENL.scale.gz --outFileNameMatrix ./3.delta_heatmap_piecharts/HK2_T1_ENL.scale.txt 


awk 'BEGIN{OFS="\t"}{if (NR > 1) print $1, $2, $3,NR-2,".", "+"}' ./2.Re_do_with_lambda_macs/HK2_T2_WT.merge.sig.anno.pval.txt > ./3.delta_heatmap_piecharts/T2.bed
****
computeMatrix reference-point --referencePoint center -b 5000 -a 5000 -bs 100 --missingDataAsZero -R ./3.delta_heatmap_piecharts/T2.bed -S /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T2_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_ENL_treat.n

> Split the group according to the previous f'{sigpath}293_{treat}_WT.merge.sig.anno.pval.txt

In [26]:
def average_line_per_colname_center(colname, colnames, line):
    eles = line.split('\t')
    vals = [
        eles[id] for id, icolname in enumerate(colnames) if icolname == colname
    ]
    N = len(vals)
    return np.mean(np.array(vals)[N // 4:N // 4 * 3].astype(float))

> Use the computeMatrxi to extract the signal and then rank the signal by upper, lower and nodiff
Which is Bad (removed in future)

In [34]:
for treat in treats:
    #5. calculate the averaged signal across all the samples
    with open(
            '{}.scale.txt'.format(outpath + exp + '_' + treat + '_' + ctrl), 'r') as f:
        print('preprocess --> {}.scale.txt'.format(outpath + exp + '_' + treat + '_' + ctrl))
        lines = f.readlines()
        lines = lines[2:]
        colnames = lines[0].rstrip().split('\t')[1:]
        colnames_uniq = np.unique(colnames)
        idx = [
            i for col in [treat, ctrl]
            for i, ele in enumerate(colnames_uniq) if col in ele
        ]
        colnames_uniq = colnames_uniq[idx]
        all_cols = []
        for colname_uniq in colnames_uniq:
            one_col = []
            for line in lines[1:]:
                one_col.append(
                    average_line_per_colname_center(
                        colname_uniq, colnames, line))
            all_cols.append(one_col)
        df = pd.DataFrame(np.array(all_cols).T, columns=colnames_uniq)
        
        # combine the peak location and signal
        bed_file_format = f'{outpath}{treat}.bed'
        df_anno = pd.read_csv(
            bed_file_format,
            sep='\t',
            header=None,
            names=['chr', 'start', 'end', 'gene', 'info', 'strand'])
        df_comb = pd.concat([df_anno, df], axis=1)
        # remove the peaks which does not have significant signal two fold more than the ctrl
        # df_sub = df_comb[(df_comb.iloc[:,3] + df_comb.iloc[:,4]) / df_comb.iloc[:,5] > 4]
        df_sub = df_comb
        #6. divide the peaks according to the enrichment into three groups
        df_sub_upper = df_sub[
            df_sub.iloc[:, -2].values / df_sub.iloc[:, -1].values > 1.5]
        df_sub_lower = df_sub[
            df_sub.iloc[:, -2].values / df_sub.iloc[:, -1].values < 1 / 1.5]
        df_sub_nodiff = df_sub[
            (df_sub.iloc[:, -2].values / df_sub.iloc[:, -1].values >= 1 / 1.5)
            & (df_sub.iloc[:, -2].values / df_sub.iloc[:, -1].values <= 1.5)]
        #7. save the upper lower and nodiff results as tables
        df_sub_upper.to_csv(
            '{}_{}_{}.average.upper.txt'.format(outpath+exp, treat, ctrl),
            index=True,
            sep='\t')
        df_sub_lower.to_csv(
            '{}_{}_{}.average.lower.txt'.format(outpath+exp, treat, ctrl),
            index=True,
            sep='\t')
        df_sub_nodiff.to_csv(
            '{}_{}_{}.average.nodiff.txt'.format(outpath+exp, treat, ctrl),
            index=True,
            sep='\t')        

preprocess --> ./res_delta_heatmap/293_T1_ENL.scale.txt
preprocess --> ./res_delta_heatmap/293_T2_ENL.scale.txt
preprocess --> ./res_delta_heatmap/293_T3_ENL.scale.txt
preprocess --> ./res_delta_heatmap/293_Y78A_ENL.scale.txt


In [ ]:
for treat in treats:
    #1. read the scale file which stores all the bins' signal
    count_file = '{}.scale.txt'.format(outpath + exp + '_' + treat + '_' + ctrl)
    df = pd.read_csv(count_file, sep='\t', header=2)
    # remove the first colname, which denotes how many peaks
    colnames = df.columns.values[1:]
    df = df.iloc[:, :-1]
    df.columns = colnames
    #2. reshape df into 3 groups with increasing order
    # *.scale.txt: data for heatmap
    # *.average.upper.txt: average value for scale and used for sorting
    print('preprocess on {}_{}_{}.scale.txt'.format(exp, treat, ctrl))
    exts = ['upper', 'lower', 'nodiff']
    for ext in exts:
        # index_col: use the previous columns info for the reads counts in df file
        df_tmp = pd.read_csv(
            '{}_{}_{}.average.{}.txt'.format(outpath+exp, treat, ctrl, ext),
            sep='\t',
            index_col=0)
        treat_id = [
            i for i, ele in enumerate(df_tmp.columns) if treat in ele
        ][0]
        df_tmp.sort_values(
            by=df_tmp.columns[treat_id], inplace=True)  #4: treat
        #3. split the df_tmp into three groups for treat and ctrl and delta
        inds = df_tmp.index
        #3.1 treat: # add 1
        treat_id = [i for i, ele in enumerate(colnames) if treat in ele]
        df_treat = df.iloc[inds, treat_id]
        df_treat.to_csv(
            '{}_{}_{}.{}.heatmap.{}.txt'.format(outpath + exp, treat,
                                                ctrl, ext, treat),
            index=False)
        #3.2 ctrl:
        ctrl_id = [i for i, ele in enumerate(colnames) if ctrl in ele]
        df_ctrl = df.iloc[inds, ctrl_id]
        df_ctrl.to_csv(
            '{}_{}_{}.{}.heatmap.{}.txt'.format(outpath + exp, treat,
                                                ctrl, ext, ctrl),
            index=False)
        #3.3 delta verified
        df_delta = pd.DataFrame(df_treat.values - df_ctrl.values)
        df_delta.to_csv(
            '{}_{}_{}.{}.heatmap.{}.txt'.format(outpath + exp, treat,
                                                ctrl, ext, 'delta'),
            index=False)

# Alternative use the average peak signal as the rank list order for the delta heatmap

In [6]:
fc = 1.5
for treat in treats:
    sigfile = f'{sigpath}{exp}_{treat}_WT.merge.sig.anno.pval.txt'
    df = pd.read_csv(sigfile,sep='\t',header=0)  
    print('preprocess on {}'.format(sigfile))
    exts = ['upper', 'lower', 'nodiff']
    ind_up = df.iloc[:,4].div(df.iloc[:,5]) >= fc
    ind_down = df.iloc[:,4].div(df.iloc[:,5]) <= (1.0/fc)
    ind_no = (df.iloc[:,4].div(df.iloc[:,5]) > 1.0/fc) & (df.iloc[:,4].div(df.iloc[:,5]) < fc)
    inds = [ind_up, ind_down, ind_no]
    # the extracted binning file 
    scalefile = '{}.scale.txt'.format(outpath + exp + '_' + treat + '_' + ctrl)
    df_scale = pd.read_csv(scalefile, sep='\t', header=2)
    # remove the first colname, which denotes how many peaks
    colnames = df_scale.columns.values[1:]
    df_scale = df_scale.iloc[:, :-1]
    df_scale.columns = colnames
    
    for ind, ext in zip(inds, exts):
        print(ext + '***\n')
        df_tmp = df[ind].copy()
        treat_id = [
            i for i, ele in enumerate(df_tmp.columns) if treat in ele
        ][0]
        df_tmp.sort_values(
            by=df_tmp.columns[treat_id], inplace=True)  #4: treat
        #3. split the df_tmp into three groups for treat and ctrl and delta
        inds = df_tmp.index
        #3.1 treat: # add 1
        treat_id = [i for i, ele in enumerate(colnames) if treat in ele]
        df_treat = df_scale.iloc[inds, treat_id]
        df_treat.to_csv(
            '{}_{}_{}.{}.heatmap.{}.txt'.format(outpath + exp, treat,
                                                ctrl, ext, treat),
            index=False)
        #3.2 ctrl:
        ctrl_id = [i for i, ele in enumerate(colnames) if ctrl in ele]
        df_ctrl = df_scale.iloc[inds, ctrl_id]
        df_ctrl.to_csv(
            '{}_{}_{}.{}.heatmap.{}.txt'.format(outpath + exp, treat,
                                                ctrl, ext, ctrl),
            index=False)
        #3.3 delta verified
        df_delta = pd.DataFrame(df_treat.values - df_ctrl.values)
        df_delta.to_csv(
            '{}_{}_{}.{}.heatmap.{}.txt'.format(outpath + exp, treat,
                                                ctrl, ext, 'delta'),
            index=False)
    

preprocess on ./2.Re_do_with_lambda_macs/HK2_T1_WT.merge.sig.anno.pval.txt
upper***

lower***

nodiff***

preprocess on ./2.Re_do_with_lambda_macs/HK2_T2_WT.merge.sig.anno.pval.txt
upper***

lower***

nodiff***

preprocess on ./2.Re_do_with_lambda_macs/HK2_T3_WT.merge.sig.anno.pval.txt
upper***

lower***

nodiff***

preprocess on ./2.Re_do_with_lambda_macs/HK2_Y78A_WT.merge.sig.anno.pval.txt
upper***

lower***

nodiff***



In [11]:
!ls ./3.delta_heatmap_piecharts/*.heatmap*.txt
!tar -cvf HK2_delta_1.5.tar ./3.delta_heatmap_piecharts/*.heatmap*.txt
!ls *.tar

./3.delta_heatmap_piecharts/HK2_T1_ENL.lower.heatmap.ENL.txt
./3.delta_heatmap_piecharts/HK2_T1_ENL.lower.heatmap.T1.txt
./3.delta_heatmap_piecharts/HK2_T1_ENL.lower.heatmap.delta.txt
./3.delta_heatmap_piecharts/HK2_T1_ENL.nodiff.heatmap.ENL.txt
./3.delta_heatmap_piecharts/HK2_T1_ENL.nodiff.heatmap.T1.txt
./3.delta_heatmap_piecharts/HK2_T1_ENL.nodiff.heatmap.delta.txt
./3.delta_heatmap_piecharts/HK2_T1_ENL.upper.heatmap.ENL.txt
./3.delta_heatmap_piecharts/HK2_T1_ENL.upper.heatmap.T1.txt
./3.delta_heatmap_piecharts/HK2_T1_ENL.upper.heatmap.delta.txt
./3.delta_heatmap_piecharts/HK2_T2_ENL.lower.heatmap.ENL.txt
./3.delta_heatmap_piecharts/HK2_T2_ENL.lower.heatmap.T2.txt
./3.delta_heatmap_piecharts/HK2_T2_ENL.lower.heatmap.delta.txt
./3.delta_heatmap_piecharts/HK2_T2_ENL.nodiff.heatmap.ENL.txt
./3.delta_heatmap_piecharts/HK2_T2_ENL.nodiff.heatmap.T2.txt
./3.delta_heatmap_piecharts/HK2_T2_ENL.nodiff.heatmap.delta.txt
./3.delta_heatmap_piecharts/HK2_T2_ENL.upper.heatmap.ENL.txt
./3.delta_hea

In [14]:
def generate_outer_links(files):
    public_url_pre = "http://dldcc-web.brc.bcm.edu/lilab/xc3/ENL2/ChIP/tmp/"
    public_url_folder = "/mount/weili2/lilab/xc3/ENL2/ChIP/tmp/"
    for ifile in files:
        file_name = ifile.split('/')[-1]
        line = f"cp {ifile} {public_url_folder}{ifile}"
        print(line)
        os.system(line)
        print(f'{public_url_pre}{ifile}')
        
files = glob.glob('*.tar')
files
generate_outer_linksrate_outer_links(files)

cp HK2_delta_1.5.tar /mount/weili2/lilab/xc3/ENL2/ChIP/tmp/HK2_delta_1.5.tar
http://dldcc-web.brc.bcm.edu/lilab/xc3/ENL2/ChIP/tmp/HK2_delta_1.5.tar


## Alternative plotting the delta heatmap using the previous generated signal to rank

In [69]:
!head $sigfile
!wc -l $sigfile
scalefile = '{}.scale.txt'.format(outpath + exp + '_' + treat + '_' + ctrl)
!wc -l $scalefile
df_scale = pd.read_csv(scalefile, sep='\t', header=2)
colnames = df_scale.columns.values[1:]
df_scale = df_scale.iloc[:, :-1]
df_scale.columns = colnames
df_tmp = df[ind_up]
treat_id = [
    i for i, ele in enumerate(df_tmp.columns) if treat in ele
][0]
df_tmp.sort_values(
    by=df_tmp.columns[treat_id], inplace=True)  #4: treat
#3. split the df_tmp into three groups for treat and ctrl and delta
inds = df_tmp.index
#3.1 treat: # add 1
treat_id = [i for i, ele in enumerate(colnames) if treat in ele]
df_treat = df.iloc[inds, treat_id]
df_treat.to_csv(
    '{}_{}_{}.{}.heatmap.{}.txt'.format(heatpath + exp, treat,
                                        ctrl, ext, treat),
    index=False)


chr	start	end	width	293_F_Y78A_treat	293_F_ENL_treat	293_ctrl	within_genebody	nearest_TSS	distance	logFC	PValue
chr1	858702	861593	2891	14.835005188516085	23.870632998962297	3.969906606710482	SAMD11	SAMD11,LOC100130417	473,-3630	-0.6859999999999999	0.552
chr1	870327	871455	1128	7.6436170212765955	12.20035460992908	3.0585106382978724	SAMD11	none	none	-0.675	0.5720000000000001
chr1	875037	878559	3522	12.062464508801815	15.90005678591709	3.0011357183418514	SAMD11	none	none	-0.39899999999999997	0.915
chr1	901715	902881	1166	9.704116638078904	10.452830188679245	2.834476843910806	PLEKHN1	PLEKHN1	-161	-0.107	0.415
chr1	932422	936620	4198	13.082896617436873	20.179132920438303	3.564078132444022	HES4	HES4	-1068	-0.625	0.6559999999999999
chr1	936657	937855	1198	11.450751252086814	10.560934891485811	3.6176961602671116	none	HES4	-1105	0.11699999999999999	0.175
chr1	955250	956213	963	8.401869158878505	11.490134994807892	3.273104880581516	AGRN	AGRN	-252	-0.452	0.982
chr1	1709153	1710245	1092	8.689560

## Heatmap

For 4, it is actually the big heatmap including all peaks called in WT, T1, T2, T3. This is to show that overall the wild type ENL and mutants bind to the same locus. It is similar to the old file I attached here “All in 293.tss.pdf”, but we need it to be peak centered, plus/minus 5 kb region. No cutoff setting applies here, any peaks called in one of the four samples (WT, T1, T2 and T3) should be included here. I think you probably need to merge peaks for this heatmap? And just plot Y78A for the same regions and put at the end, no need to include Y78A when merging peaks.

> merge all the peaks together

In [16]:
# merge a list of the bed files
# depeand on the bedtools
def merge_beds(bed_files, out_name, header=True):
    """
    bed_files: a list of bed files to be merged
    """
    lines = []
    for bed_file in bed_files:
        with open(bed_file, 'r') as f:
            lines.extend(f.readlines()[1:] if header else f.readlines())
    # merge the two beds together        
    with open('tmp.bed','w') as f:
        f.writelines(lines)
    
    # sort the files
    command = "sort -k1,1 -k2,2n tmp.bed > tmp.sorted.bed"
    print(f'execute {command}')
    os.system(command)
    
    # merge the files
    command = f"bedtools merge -i tmp.sorted.bed -d 1 > {out_name}"
    print(f'execute {command}')
    os.system(command)

In [19]:
!mkdir -p ./3.heatmap_union

1.Use_first_batch_coup_extract_signal_from_second_batch.ipynb
2.Re_do_with_lambda_macs
2.Re_do_with_lambda_macs.ipynb
3.delta_heatmap_piecharts
3.do_delta_heatmap_piecharts.ipynb
3.heatmap_union
avg_sig_0.txt
avg_sig_1.txt
avg_sig_2.txt
de_pval_counts.sh
heatmap_union
res_second_batch_using_first_batch_peak
testT1WT.bed
testY78AWT.bed
tmp.bed
tmp.sorted.bed


In [21]:
os.chdir('/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/3.heatmap_union/')
!pwd
peakpath = '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/'

/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/3.heatmap_union


In [23]:
peaksbed = glob.glob(peakpath + 'HK2_F_*.nsp_peaks.bed')
peaksbed.sort()
peaksbed = peaksbed[:-1]
peaksbed

['/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.bed',
 '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T1.nsp_peaks.bed',
 '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T2.nsp_peaks.bed',
 '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T3.nsp_peaks.bed']

In [24]:
print(peaksbed)
out_name = f'HK2_treat_WT.union.merge.bed'
merge_beds(peaksbed, out_name=out_name, header=False)

['/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.bed', '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T1.nsp_peaks.bed', '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T2.nsp_peaks.bed', '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T3.nsp_peaks.bed']
execute sort -k1,1 -k2,2n tmp.bed > tmp.sorted.bed
execute bedtools merge -i tmp.sorted.bed -d 1 > HK2_treat_WT.union.merge.bed


In [27]:
# computing the tss signal
exp = 'HK2'
bwfiles = glob.glob(bwpath+exp+'*.bw')
bwfiles.sort()
bwfiles = bwfiles[:5]
name = 'TsWT'
profile_bed = out_name
line = 'computeMatrix reference-point --referencePoint center -S {} -R {} --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o {}.Ts.ENL.center.gz -p 8 --sortUsingSamples 1 --outFileSortedRegions {}.sorted.bed'.format(' '.join(bwfiles), profile_bed, '{}'.format(name), name)
print(line)
os.system(line)

computeMatrix reference-point --referencePoint center -S /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_ENL_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T1_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T2_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T3_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_Y78A_treat.nsp.bw -R HK2_treat_WT.union.merge.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o TsWT.Ts.ENL.center.gz -p 8 --sortUsingSamples 1 --outFileSortedRegions TsWT.sorted.bed


0

In [28]:
!ls

HK2_treat_WT.union.merge.bed  TsWT.sorted.bed  tmp.sorted.bed
TsWT.Ts.ENL.center.gz	      tmp.bed


In [36]:
# plot the heatmap for the union of the beds
line = 'plotHeatmap --sortUsingSamples 1 -m TsWT.Ts.ENL.center.gz  -out HK2.{}.center.union.pdf --outFileNameMatrix {} --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 10 --refPointLabel center'.format(name, name, name+'.heatmap.center.union.mat')
print(line)
os.system(line)

plotHeatmap --sortUsingSamples 1 -m TsWT.Ts.ENL.center.gz  -out HK2.TsWT.center.union.pdf --outFileNameMatrix TsWT --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 10 --refPointLabel center


0

## generated the heatmap associated excel file

In [32]:
import gzip
import re

In [33]:
def convert_deeptools_gz_to_df(file_name):
    """
    file_name: the deeptools computeMatrix output matrix gz file
    caculate the avg signal per sample across all the bins
    """
    with gzip.open(file_name, 'rt', encoding='utf-8') as f:
        lines = f.readlines()
    line = lines[0]
    a = line.split('@')[-1].split('\\')[0]
    a = re.sub('true', 'True', a)
    a = re.sub('false', 'False', a)
    a = re.sub('null', 'False', a)    
    info = eval(a)
    with open(f'{file_name}.txt', 'w') as f:
        f.writelines(lines[1:])

    df = pd.read_csv(f'{file_name}.txt', sep='\t', header=None) 
    for i, sample in enumerate(info['sample_labels']):
        start = info['sample_boundaries'][i] + 6
        end = info['sample_boundaries'][i+1] + 6    
        df[sample] = df.iloc[:, start:end].mean(axis=1)

    df.sort_values(by=[info['sample_labels'][0]], inplace=True, ascending=False)     
    return df

In [34]:
df = convert_deeptools_gz_to_df(file_name = f'TsWT.Ts.ENL.center.gz')
df.iloc[:, [0,1,2,-5,-4,-3,-2,-1]].to_csv('HK2.TsWT.Ts.ENL.center.sig.txt', index=False, sep='\t')

In [35]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,501,502,503,504,505,HK2_F_ENL_treat.nsp,HK2_F_T1_treat.nsp,HK2_F_T2_treat.nsp,HK2_F_T3_treat.nsp,HK2_F_Y78A_treat.nsp
8928,chr21,9825287,9828084,chr21:9825287-9828084,.,.,0.00,0.00,0.00,0.00,...,5.20,4.85,3.20,2.55,7.15,72.4015,73.8760,71.9365,75.5500,73.8525
4025,chr14,62160514,62174448,chr14:62160514-62174448,.,.,111.75,122.99,109.76,113.44,...,6.61,7.61,13.26,16.40,15.29,48.6380,62.7270,46.0369,73.6199,12.8394
4073,chr14,69250575,69263613,chr14:69250575-69263613,.,.,19.84,20.34,17.72,19.26,...,12.60,13.64,11.12,9.40,10.66,43.2216,31.9252,32.7894,53.8662,12.8782
12397,chr7,27185065,27221024,chr7:27185065-27221024,.,.,33.32,43.68,53.78,66.28,...,9.24,10.02,7.32,5.68,13.92,42.5946,125.1380,84.1482,129.1008,11.2298
9801,chr3,101565533,101584291,chr3:101565533-101584291,.,.,62.66,60.32,49.18,43.00,...,13.28,8.98,6.36,6.44,7.60,41.7046,42.0786,28.6024,59.6928,11.2794


# Do the pie charts on the local lambda

BTW, have you generated pie charts for this batch of new peak lists?

In [37]:
os.chdir('/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/3.delta_heatmap_piecharts/')
!pwd

/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/3.delta_heatmap_piecharts


In [38]:
peaksbed = glob.glob(peakpath + f'{exp}_F_*.nsp_peaks.bed')
peaksbed.sort()
peaksbed

['/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.bed',
 '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T1.nsp_peaks.bed',
 '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T2.nsp_peaks.bed',
 '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T3.nsp_peaks.bed',
 '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_Y78A.nsp_peaks.bed']

In [40]:
def annoate_bed_to_gene(infile, outfile, cutoff=5000):
    ref, tss_set = {}, set()
    reffile = '/mount/weili3/xc3/genomes/hg19.refGene.txt'
    for line in open(reffile):
        col = line.split('\t')
        name, cr, strand, TSS, TES, symbol = col[1], col[2], col[3], int(col[4]), int(col[5]), col[12]
        if strand == '-': TSS, TES = TES, TSS
        if cr not in ref: ref[cr] = []
        if (cr,TSS,strand) not in tss_set:
            ref[cr].append((name,symbol,strand,TSS,TES))
            tss_set.add((cr,TSS,strand))
    for cr in ref: ref[cr].append(('none','none','none',0,0))
    
    # annotate the file
    text = open(infile).readlines()
    fout = open(outfile, 'w')
    print('processing on {}\n will output {}\n'.format(infile,outfile))
    for line in text:
        col = line.split('\t')
        try: cr, start, end = col[0], int(col[1]), int(col[2])
        except: 
            fout.write(line[:-1]+'\twithin_genebody\tnearest_TSS\tdistance\n')
            continue
        if cr not in ref: continue
        peak = (start + end) / 2
        genes, genebody, genes0, genebody0 = [], [], [], []
        for name, symbol, strand, TSS, TES in ref[cr]:
            if strand == '+':
                dist = end - TSS
                if abs(start-TSS)<abs(dist):dist=start-TSS

            elif strand == '-':
                dist = TSS - end
                if abs(TSS - start)<abs(dist):dist=TSS - start

            elif strand != 'none': raise ValueError
            if abs(dist) <= cutoff: genes.append((abs(dist),symbol,dist)) 
            if (start - TSS) * (start - TES) <= 0: genebody.append((abs(dist),symbol,dist))
            elif (end - TSS) * (end - TES) <= 0: genebody.append((abs(dist),symbol,dist))
            elif (start - TSS) * (end - TES) <=0: genebody.append((abs(dist),symbol,dist))
        genes, genebody = sorted(genes), sorted(genebody)
        for g in sorted(genes):
            if g[1] not in [x[1] for x in genes0]: genes0.append(g)
        for g in sorted(genebody):
            if g[1] not in [x[1] for x in genebody0]: genebody0.append(g)
        if any(genes0):
            symbols = ','.join([x[1] for x in genes0])
            dists = ','.join(['%d' % x[2] for x in sorted(genes0)])
        else: symbols, dists = 'none', 'none'
        if any(genebody): body = ','.join([x[1] for x in genebody0])
        else: body = 'none'
        fout.write(line[:-1] + '\t%s\t%s\t%s\n' % (body,symbols,dists))
    fout.close()   

In [49]:
for infile in peaksbed:
    outfile = infile.split('.bed')[0] + '.3k.anno.txt'
    annoate_bed_to_gene(infile, outfile,3000)

processing on /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.bed
 will output /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.3k.anno.txt

processing on /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T1.nsp_peaks.bed
 will output /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T1.nsp_peaks.3k.anno.txt

processing on /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T2.nsp_peaks.bed
 will output /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T2.nsp_peaks.3k.anno.txt

processing on /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T3.nsp_peaks.bed
 will output /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T3.nsp_peaks.3k.anno.txt

processing on /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_Y78A.nsp_peaks.bed
 will output /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_Y78A.nsp_peaks.3k.anno.txt



In [50]:
files = glob.glob(peakpath + f'{exp}*.3k.anno.txt')
files

['/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.3k.anno.txt',
 '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T1.nsp_peaks.3k.anno.txt',
 '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T2.nsp_peaks.3k.anno.txt',
 '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T3.nsp_peaks.3k.anno.txt',
 '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_Y78A.nsp_peaks.3k.anno.txt']

In [57]:
names = [ifile.split('/')[-1].split('.')[0] for ifile in files]
for i in range(len(files)):
    file2plot = files[i]
    outfile = '{}.pie.pdf'.format(names[i])
    line = 'python ~/xc3/software/plot_piechart.py {} {}'.format(file2plot, './')
    print(line)
    os.system(line)

python ~/xc3/software/plot_piechart.py /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.3k.anno.txt ./
python ~/xc3/software/plot_piechart.py /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T1.nsp_peaks.3k.anno.txt ./
python ~/xc3/software/plot_piechart.py /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T2.nsp_peaks.3k.anno.txt ./
python ~/xc3/software/plot_piechart.py /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T3.nsp_peaks.3k.anno.txt ./
python ~/xc3/software/plot_piechart.py /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_Y78A.nsp_peaks.3k.anno.txt ./


## count the peaks number for piecharts

In [62]:
for infile in files:
#     print(infile)
    !wc -l $infile

9911 /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.3k.anno.txt
7154 /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T1.nsp_peaks.3k.anno.txt
8010 /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T2.nsp_peaks.3k.anno.txt
13710 /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T3.nsp_peaks.3k.anno.txt
4539 /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_Y78A.nsp_peaks.3k.anno.txt
